In [63]:
import pandas as pd
import vk
import time

session = vk.AuthSession('6212339', '89817730298', '162445mda618')
vk_api = vk.API(session)

pd.set_option('display.max_columns', 500)

In [21]:
#Необходимо получить список друзей всех уникальных пользователей из кластера

users_from_136_clusters_df = pd.read_csv('../Data/ProcesData/users_from_136_clusers.csv', encoding='windows-1251')

In [75]:
#return 'code' parameter for VK API execute method
#Param
#users_id_list - bach of 25 users id

def get_user_friends_vk_code(users_id_list):
    code = 'return {'
    
    for id in users_id_list:
        code = '%s%s' % (code, '"%s": API.friends.get({"user_id":%s}),' % (id, id))
    
    code = '%s%s' % (code, '};')
        
    return code

def parts(lst, n=25):
    return [lst[i:i + n] for i in iter(range(0, len(lst), n))]

def find_users(user_id_bachs):
    users = []
    for bach in user_id_bachs:
        try:
            users_list = vk_api.users.get(user_ids=bach, v=5.80)
        except Exception as e:
            print(e)
            
            continue
            
        users.extend(users_list)
        
    return users 

def get_users_friends(users_id_list):
    users_id_bachs = parts(users_id_list, 25)
    
    users_friends_dict = {}
    for bach in users_id_bachs:
        try:
            u_f_l = vk_api.execute(code=get_user_friends_vk_code(bach), v=5.80)
            
            time.sleep(1)
            
        except Exception as e:
            print(e)
            continue
        
        users_friends_dict.update(u_f_l)
    
    return users_friends_dict

In [55]:
#delete deactivated users
users_id_bachs = parts(list(map(lambda u: int(u), users_from_136_clusters_df.uid.tolist())),1000)
users_list = find_users(users_id_bachs)

users_136_info_list = pd.DataFrame(users_list)
users_136_info_list = users_136_info_list.rename(index=str, columns={"id": "uid"})

users_from_136_clusters_df = users_from_136_clusters_df[['uid']].merge(users_136_info_list, on='uid', how='inner')

users_from_136_clusters_df['uid'] = users_from_136_clusters_df.uid.apply(lambda u: int(u))
users_from_136_clusters_df = users_from_136_clusters_df[users_from_136_clusters_df.deactivated.isnull()]

In [76]:
#get friends
users_friends = get_users_friends(users_from_136_clusters_df.uid.tolist())

In [87]:
uid_list = list(users_friends.keys())
friends_count = list(map(lambda u: u['count'], list(users_friends.values())))
friends_list = list(map(lambda u: u['items'], list(users_friends.values())))

In [88]:
users_friends_df = pd.DataFrame({'uid':uid_list, 'friends_count':friends_count, 'friends_list': friends_list})

In [90]:
users_friends_df.to_csv('../Data/ProcesData/users_friends.csv')